<a href="https://colab.research.google.com/github/Chumunaca/Chumunaca/blob/main/DETECTRON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ==========================================
# 1. INSTALACIÓN DE DEPENDENCIAS
# ==========================================
print("🚀 --- FASE 1: INSTALANDO DETECTRON2 ---")
# Removed explicit pyyaml==5.1 installation as it failed and detectron2 from source will handle its own pyyaml dependencies.

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print(f"   -> Detectado: PyTorch {TORCH_VERSION} | CUDA {CUDA_VERSION}")

# Instalación compatible con Colab
# Reemplazar la instalación basada en wheels con la instalación desde el repositorio de GitHub.
# Esto es más robusto en Colab ya que construye Detectron2 contra las versiones existentes de PyTorch/CUDA.
print(f"   -> Instalando Detectron2 desde el código fuente (para compatibilidad con PyTorch {TORCH_VERSION} | CUDA {CUDA_VERSION})...")
!pip install -q 'git+https://github.com/facebookresearch/detectron2.git'

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
print("✅ Detectron2 instalado.")

# ==========================================
# 2. DESCOMPRESIÓN DE DATOS (AL DISCO RÁPIDO)
# ==========================================
import os

print("\n🚀 --- FASE 2: MOVIMIENTO DE DATOS ---")
# Rutas en Drive
ZIP_PATH_DRIVE = '/content/drive/MyDrive/TESIS/imagenes/vindr.zip'
CSV_PATH_DRIVE = '/content/drive/MyDrive/TESIS/TESIS/finding_annotations.csv'

# Ruta Local (Rápida)
EXTRACT_DIR = '/content/vindr_data'
IMG_BASE_DIR = os.path.join(EXTRACT_DIR, 'images_png')

if not os.path.exists(IMG_BASE_DIR):
    print(f"   -> Descomprimiendo {ZIP_PATH_DRIVE} en local...")
    os.makedirs(EXTRACT_DIR, exist_ok=True)
    # Unzip rápido
    !unzip -q "{ZIP_PATH_DRIVE}" -d "{EXTRACT_DIR}"
    print("✅ Descompresión terminada. Imágenes listas en local.")
else:
    print("✅ Las imágenes ya estaban en el disco local.")

# ==========================================
# 3. PREPARACIÓN Y ESCALADO (La Regla de 3)
# ==========================================
import pandas as pd
import numpy as np
from detectron2.structures import BoxMode
from detectron2.data import DatasetCatalog, MetadataCatalog
from ast import literal_eval
from tqdm import tqdm

print("\n🚀 --- FASE 3: REGISTRO DE DATASET ---")

# TUS DIMENSIONES OBJETIVO
TARGET_H = 1520
TARGET_W = 912

# Cargar CSV
df = pd.read_csv(CSV_PATH_DRIVE)
df = df[df['xmin'].notna()].copy() # Solo lesiones

# Limpiar categorías
def clean_cat(x):
    try:
        if isinstance(x, str):
            l = literal_eval(x)
            return l[0] if l else "Unknown"
    except:
        return x
    return x

df['category'] = df['finding_categories'].apply(clean_cat)
classes = sorted(df['category'].unique())
class_to_id = {name: i for i, name in enumerate(classes)}
print(f"   -> Clases encontradas: {classes}")

def get_vindr_dicts(split):
    dataset_dicts = []
    data_split = df[df['split'] == split]
    grouped = data_split.groupby('image_id')

    print(f"   -> Procesando split: {split}...")
    for img_id, group in tqdm(grouped):
        study_id = group.iloc[0]['study_id']
        # Ruta local rápida
        file_path = os.path.join(IMG_BASE_DIR, study_id, f"{img_id}.png")

        if not os.path.exists(file_path): continue

        # Dimensiones ORIGINALES (del CSV)
        orig_h = int(group.iloc[0]['height'])
        orig_w = int(group.iloc[0]['width'])

        # Factores de escala
        scale_x = TARGET_W / orig_w
        scale_y = TARGET_H / orig_h

        record = {}
        record["file_name"] = file_path
        record["image_id"] = img_id
        record["height"] = TARGET_H
        record["width"] = TARGET_W

        objs = []
        for _, row in group.iterrows():
            cat = row['category']
            if cat not in class_to_id: continue

            # ESCALADO DE CAJAS
            x1 = row['xmin'] * scale_x
            y1 = row['ymin'] * scale_y
            x2 = row['xmax'] * scale_x
            y2 = row['ymax'] * scale_y

            obj = {
                "bbox": [x1, y1, x2, y2],
                "bbox_mode": BoxMode.XYXY_ABS,
                "category_id": class_to_id[cat],
            }
            objs.append(obj)

        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

# Registrar
DatasetCatalog.clear()
MetadataCatalog.clear()
for d in ["training", "test"]:
    DatasetCatalog.register(f"vindr_{d}", lambda x=d: get_vindr_dicts(x))
    MetadataCatalog.get(f"vindr_{d}").set(thing_classes=classes)

print("✅ Dataset registrado y escalado correctamente.")

In [ ]:
import random
import cv2
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer

# Agarrar la metadata de lo que acabamos de registrar
vindr_metadata = MetadataCatalog.get("vindr_training")

#Traer los diccionarios (la lista de imágenes y cajas)
dataset_dicts = get_vindr_dicts("training")

print("🎲 Agarrando 3 imágenes al azar para ver si las cajas coinciden con las lesiones...")

# Visualizar 3 muestras
for d in random.sample(dataset_dicts, 3):
    # Leer imagen
    img = cv2.imread(d["file_name"])

    # Dibujar las cajas "Ground Truth" (La Verdad Absoluta)
    visualizer = Visualizer(img[:, :, ::-1], metadata=vindr_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)

    # Mostrar con Matplotlib
    plt.figure(figsize=(10, 12))
    plt.imshow(out.get_image())
    plt.title(f"Imagen: {d['image_id']}\nCajas Reales (Entrenamiento)")
    plt.axis('off')
    plt.show()

In [ ]:
import os
import shutil
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# --- 1. LIMPIEZA (Borrón y Cuenta Nueva) ---
OUTPUT_DIR = "/content/drive/MyDrive/TESIS/detectron_output_vindr_A100_HEAVY"

if os.path.exists(OUTPUT_DIR):
    print(f"🗑️ Borrando modelo anterior en {OUTPUT_DIR}...")
    shutil.rmtree(OUTPUT_DIR) # Adiós basura
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✨ Directorio limpio y listo: {OUTPUT_DIR}")

# --- 2. CONFIGURAR LA BESTIA (X101) ---
print("⚙️ Cargando configuración de ResNeXt-101 (The Beast)...")
cfg = get_cfg()

# Usamos X_101_32x8d_FPN_3x (Mucho más poderoso que R_50)
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))

# Datos
cfg.DATASETS.TRAIN = ("vindr_training",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 4

# Pesos Iniciales
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")

# --- 3. HIPERPARÁMETROS AGRESIVOS PARA A100 ---
# X101 consume más VRAM, así que ajustamos el batch
cfg.SOLVER.IMS_PER_BATCH = 6   # 6 imágenes gigantes por batch (A100 aguanta)
cfg.SOLVER.BASE_LR = 0.001     # Learning Rate
cfg.SOLVER.WARMUP_ITERS = 1000 # Calentamiento suave para no romper gradientes

# ¡AQUI ESTÁ EL CAMBIO FUERTE! -> MÁS TIEMPO
# Si 2000 iters fueron 15 min...
# 10,000 iters serán como 1 hora y cachito. ¡DALE SIN MIEDO!
cfg.SOLVER.MAX_ITER = 10000

cfg.SOLVER.STEPS = (7000, 9000) # Bajamos el LR al final para pulir
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 # Más muestras por imagen (más fino)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes) # Tus clases

# Umbral de prueba (para que cuando termine, guarde esto en el config)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2

cfg.OUTPUT_DIR = OUTPUT_DIR

# --- 4. ENTRENAR ---
print(f"🔥 ARRANCANDO LA A100 CON X101 POR {cfg.SOLVER.MAX_ITER} ITERACIONES...")
print("Ve por cenar, echarte un sueñito o lo que quieras. Esto va a quedar perrón.")

trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

print(f"✅ ¡TERMINÓ LA BESTIA! Modelo guardado en: {cfg.OUTPUT_DIR}")

In [ ]:
from detectron2.utils.visualizer import ColorMode
import random
import cv2
import matplotlib.pyplot as plt
from detectron2.engine import DefaultPredictor

print("--- 🔮 VISUALIZANDO LA BESTIA (X101) ---")

# 1. Configurar para Inferencia
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.15 # Umbral de confianza (40%)
# Si no sale nada con 0.4, bájalo a 0.2
cfg.DATASETS.TEST = ("vindr_test", )

predictor = DefaultPredictor(cfg)

# 2. Visualizar
dataset_dicts = get_vindr_dicts("test")

# Vamos a ver 5 imágenes al azar
for d in random.sample(dataset_dicts, 5):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)

    # Dibujar predicciones
    v = Visualizer(im[:, :, ::-1],
                   metadata=MetadataCatalog.get("vindr_train"),
                   scale=0.5,
                   instance_mode=ColorMode.IMAGE_BW # B/N para contraste
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

    plt.figure(figsize=(10, 12))
    plt.imshow(out.get_image())
    plt.title(f"Predicción X101 | ID: {d['image_id']}")
    plt.axis('off')
    plt.show()

In [ ]:
import os
import shutil
from datetime import datetime

# Ruta donde se guardó el entrenamiento
OUTPUT_DIR = "/content/drive/MyDrive/TESIS/detectron_output_vindr_A100_HEAVY"
MODEL_PATH = os.path.join(OUTPUT_DIR, "model_final.pth")

if os.path.exists(MODEL_PATH):
    print(f"✅ ¡CONFIRMADO! El modelo existe en: {MODEL_PATH}")

    # Hacemos una copia de seguridad con fecha y hora (por si reentrenas por error y lo borras)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    backup_path = os.path.join(OUTPUT_DIR, f"model_final_BACKUP_{timestamp}.pth")

    shutil.copy(MODEL_PATH, backup_path)
    print(f"🔒 Copia de seguridad creada: {backup_path}")
    print("¡Ya nadie te lo borra!")
else:
    print("🚨 ¡ALERTA! No encuentro el modelo. ¿Terminó de entrenar?")

In [ ]:
import json
import matplotlib.pyplot as plt
import pandas as pd
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# --- 1. GRÁFICA DE APRENDIZAJE (LOSS CURVE) ---
# Detectron guarda todo en metrics.json
json_file = os.path.join(cfg.OUTPUT_DIR, "metrics.json")

if os.path.exists(json_file):
    data = []
    with open(json_file) as f:
        for line in f:
            data.append(json.loads(line))

    df_metrics = pd.DataFrame(data)

    # Plotear
    plt.figure(figsize=(12, 6))
    plt.plot(df_metrics['iteration'], df_metrics['total_loss'], label='Error Total (Loss)', color='red')
    if 'loss_cls' in df_metrics.columns:
        plt.plot(df_metrics['iteration'], df_metrics['loss_cls'], label='Error Clasificación', color='blue', alpha=0.5)
    if 'loss_box_reg' in df_metrics.columns:
        plt.plot(df_metrics['iteration'], df_metrics['loss_box_reg'], label='Error Coordenadas', color='green', alpha=0.5)

    plt.title('Curva de Aprendizaje: Faster R-CNN X101', fontsize=14, fontweight='bold')
    plt.xlabel('Iteraciones')
    plt.ylabel('Loss (Menos es mejor)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('grafica_loss_detectron.png') # Se guarda para que la descargues
    plt.show()
    print("✅ Gráfica de Loss generada.")
else:
    print("⚠️ No encontré el archivo metrics.json (quizás borraste la carpeta output?)")

# --- 2. EVALUACIÓN COCO (ESTÁNDAR DE ORO) ---
print("\n🔥 Corriendo Evaluación Estadística COCO (Esto tarda unos minutos)...")

# Configurar evaluador
evaluator = COCOEvaluator("vindr_test", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "vindr_test")

# Correr inferencia masiva
print("Calculando precisión por clase...")
results = inference_on_dataset(predictor.model, val_loader, evaluator)

# Imprimir bonito
print("\n" + "="*50)
print("RESULTADOS ESTADÍSTICOS FINALES (mAP)")
print("="*50)
# Esto imprime la tabla desglosada por clase (AP50, AP75, etc.)
print(results)